<a href="https://colab.research.google.com/github/ihawryluk/importations/blob/main/Thesis_double_spike_with_saving_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Renewal Equation simple model

Based on the code Sam shared on Zulip

Possibly some more useful code here https://github.com/MLGlobalHealth/uncertainity_infectious_diseases/blob/main/notebooks/SARS_COV1_inference_FINAL.ipynb



Trying to keep it simple for now, assumptions:

* fitting to infections only (not deaths)
* infections == cases
*


In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/importations/Data/Double_spike/'

In [ ]:
!pip install numpyro

In [ ]:
!pip install dill

In [ ]:
import numpyro
import jax
numpyro.set_host_device_count(4)  # 4 cores
from jax import vmap
import jax.numpy as jnp
import jax.random as random

from numpyro.infer import Predictive
import numpyro.distributions as dist
import arviz as az
from numpyro.infer import MCMC, NUTS
from numpyro.contrib.control_flow import scan

from scipy.stats import gamma
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import arviz as az
import dill

from google.colab import drive
drive.mount('/content/drive')


# 1. Simulate data

In [ ]:
def plot(R, mu, obs):
  plt.figure(figsize=(10,5))
  plt.subplot(1,2,1)
  plt.plot(obs, label = 'all cases')
  plt.plot(mu, label = 'importations')
  plt.legend()
  plt.xlabel('day')
  plt.ylabel('observed cases')
  plt.title('observed cases')
  plt.subplot(1,2,2)
  plt.plot(R, color = 'green', label='R')
  plt.xlabel('day')
  plt.ylabel('R')
  plt.title('R')
  plt.suptitle('simulated data')
  plt.tight_layout()
  plt.show()

In [ ]:
def simulate_g(n):
  g = gamma.pdf(np.arange(n), a=3, scale=1)
  g = g/np.sum(g)
  return g

# def simulate_R(n):
#   R = 1.15 + np.sin(0.15 * np.arange(n))
#   return R

def simulate_R(n, inter, mult, div=1.0):
  R = inter + np.sin(mult * np.arange(n)) / div
  return R

def simulate_mu(spikes_locs, spikes_vals):
  mu = np.zeros(n)
  mu[0] = 5  # to make sure the epidemic starts before the importation
  mu[spikes_locs] = spikes_vals
  return mu

def compute_incidence_faster(R,g,mu):
    I = jnp.ones(n)
    for i in range(n):
        # I = jax.ops.index_update(I,i,R[i]*jnp.dot(I[i:],g[:n-i]))  # deprecated
        I = I.at[i].set(R[i]*jnp.dot(I[:i],np.flip(g[:i])) + mu[i])

    return I

In [ ]:
def simulate_R(n, inter, mult, div=1.0):
  R = inter + np.sin(mult * np.arange(n)) / div
  return R

In [ ]:
n = 50
plt.plot(simulate_R(n, 1.15, 0.15))
# plt.plot(simulate_R(n, 1.5, 0.15))
plt.plot(simulate_R(n, 1.15, 0.2))
plt.plot(simulate_R(n, 1.15, 0.2, 2.0))

In [ ]:
def simulate_renewal(spikes_locs, spikes_vals, inter, mult, div=1.0, n=100):

  g = simulate_g(n)#.astype(int)   # discrete?
  mu = simulate_mu(spikes_locs, spikes_vals)
  R = simulate_R(n, inter, mult, div)

  incidence = compute_incidence_faster(R,g,mu)
  # obs = dist.Poisson(incidence).sample(random.PRNGKey(np.random.randint(10000)))
  obs = dist.Poisson(incidence).sample(random.PRNGKey(0))

  return R, incidence, g, mu, obs

In [ ]:
scenarios_names = ['placeholder',
                   r'second spike when $R_t$ decreasing, equal spikes',
                   r'second spike when $R_t$ decreasing, first spike higher',
                   r'second spike when $R_t$ decreasing, second spike higher, $\mu_2 = 150$',
                   r'second spike when $R_t$ decreasing, second spike higher, $\mu_2 = 300$',
                   r'second spike when $R_t$ increasing but inc ~ 0, equal spikes',
                   r'second spike when $R_t$ increasing but inc ~ 0, first spike higher',
                   r'second spike when $R_t$ increasing but inc ~ 0, second spike higher, $\mu_2 = 150$',
                   r'second spike when $R_t$ increasing but inc ~ 0, second spike higher, $\mu_2 = 300$',
                  r'second spike when $R_t$ increasing, equal spikes',
                   r'second spike when $R_t$ increasing, first spike higher',
                   r'second spike when $R_t$ increasing, second spike higher, $\mu_2 = 150$',
                   r'second spike when $R_t$ increasing, second spike higher, $\mu_2 = 300$'
                   ]

In [ ]:
spikes_locs1 = [5,20] # second spike when Rt decreasing
spikes_locs2 = [5,40] # second spike when Rt increasing but incidence close to 0
spikes_locs3 = [5,10] # second spike when Rt increasing


spikes_vals1 = [100, 100]
spikes_vals2 = [150, 100]
spikes_vals3 = [100, 150]
spikes_vals4 = [100, 300]

inter = 1.15
mult = 0.15
div = 1

n = 50

R1, inc1, g1, mu1, obs1 = simulate_renewal(spikes_locs1, spikes_vals1, inter, mult, div, n=n)
R2, inc2, g2, mu2, obs2 = simulate_renewal(spikes_locs1, spikes_vals2, inter, mult, div, n=n)
R3, inc3, g3, mu3, obs3 = simulate_renewal(spikes_locs1, spikes_vals3, inter, mult, div, n=n)
R4, inc4, g4, mu4, obs4 = simulate_renewal(spikes_locs1, spikes_vals4, inter, mult, div, n=n)

# R, inc, g, mu, obs = simulate_renewal(mu=None)

plot(R1, mu1, obs1)
plot(R2, mu2, obs2)
plot(R3, mu3, obs3)
plot(R4, mu4, obs4)


In [ ]:
R5, inc5, g5, mu5, obs5 = simulate_renewal(spikes_locs2, spikes_vals1, inter, mult, div, n=n)
R6, inc6, g6, mu6, obs6 = simulate_renewal(spikes_locs2, spikes_vals2, inter, mult, div, n=n)
R7, inc7, g7, mu7, obs7 = simulate_renewal(spikes_locs2, spikes_vals3, inter, mult, div, n=n)
R8, inc8, g8, mu8, obs8 = simulate_renewal(spikes_locs2, spikes_vals4, inter, mult, div, n=n)

# R, inc, g, mu, obs = simulate_renewal(mu=None)

plot(R5, mu5, obs5)
plot(R6, mu6, obs6)
plot(R7, mu7, obs7)
plot(R8, mu8, obs8)

In [ ]:
R9, inc9, g9, mu9, obs9 = simulate_renewal(spikes_locs3, spikes_vals1, inter, mult, div, n=n)
R10, inc10, g10, mu10, obs10 = simulate_renewal(spikes_locs3, spikes_vals2, inter, mult, div, n=n)
R11, inc11, g11, mu11, obs11 = simulate_renewal(spikes_locs3, spikes_vals3, inter, mult, div, n=n)
R12, inc12, g12, mu12, obs12 = simulate_renewal(spikes_locs3, spikes_vals4, inter, mult, div, n=n)

# R, inc, g, mu, obs = simulate_renewal(mu=None)

plot(R9, mu9, obs9)

## Save the simulated data

In [ ]:
# get a dataframe with R_T
Rt_df = pd.DataFrame(columns = ['R1', 'R2', 'R3', 'R3' 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12'])
Rt_df['R1'] = R1
Rt_df['R2'] = R2
Rt_df['R3'] = R3
Rt_df['R4'] = R4
Rt_df['R5'] = R5
Rt_df['R6'] = R6
Rt_df['R7'] = R7
Rt_df['R8'] = R8
Rt_df['R9'] = R9
Rt_df['R10'] = R10
Rt_df['R11'] = R11
Rt_df['R12'] = R12
# print(Rt_df)
# get a dataframe with incidence
y_df = pd.DataFrame(columns = ['y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10', 'y11', 'y12'])
y_df['y1'] = obs1
y_df['y2'] = obs2
y_df['y3'] = obs3
y_df['y4'] = obs4
y_df['y5'] = obs5
y_df['y6'] = obs6
y_df['y7'] = obs7
y_df['y8'] = obs8
y_df['y9'] = obs9
y_df['y10'] = obs10
y_df['y11'] = obs11
y_df['y12'] = obs12
# print(y_df)
# get a dataframe with mu
mu_df = pd.DataFrame(columns = ['mu1', 'mu2', 'mu3', 'mu4', 'mu5', 'mu6', 'mu7', 'mu8', 'mu9', 'mu10', 'mu11', 'mu12'])
mu_df['mu1'] = mu1
mu_df['mu2'] = mu2
mu_df['mu3'] = mu3
mu_df['mu4'] = mu4
mu_df['mu5'] = mu5
mu_df['mu6'] = mu6
mu_df['mu7'] = mu7
mu_df['mu8'] = mu8
mu_df['mu9'] = mu9
mu_df['mu10'] = mu10
mu_df['mu11'] = mu11
mu_df['mu12'] = mu12
# print(y_df)

In [ ]:
Rt_df.to_csv(path + 'sim_double_spike_Rt.csv', index=False)
y_df.to_csv(path + 'sim_double_spike_y.csv', index=False)
mu_df.to_csv(path + 'sim_double_spike_mu.csv', index=False)

In [ ]:
ls /content/drive/MyDrive/Colab\ Notebooks/importations/Data/Double_spike/

# 2. Inference

## Model

In [ ]:
def renewal(y,g,r_prior = 1.0):
# y -- observed data
# g -- generation/serial interval
    def compute_incidence_faster(R,g,mu):
        I = jnp.ones(n)
        for i in range(n):
            # I = jax.ops.index_update(I,i,R[i]*jnp.dot(I[i:],g[:n-i]))  # deprecated
            I = I.at[i].set(R[i]*jnp.dot(I[:i],np.flip(g[:i])) + mu[i])
        return I

    n = y.size

    # priors
    sigma_R = numpyro.sample("sigma_R", dist.Exponential(r_prior))   # mean of this is 1/lambda, so to make it smaller, make lambda bigger# 5 fits perf
    # sigma_R = .5  # 1 is kinda good but still not solving our problem, 0.5 is perfect

    sigma_mu = numpyro.sample("sigma_mu", dist.Gamma(.1))
    mult_mu = 1.0#numpyro.sample("mult_mu", dist.HalfNormal(4))

    theta = numpyro.sample("theta", dist.Uniform(0,1))
    temp = numpyro.sample("temp", dist.Uniform(0,1))
    bin_mu = numpyro.sample("bin_mu", dist.RelaxedBernoulli(temp, probs=theta * jnp.ones(n)))   # so seems that this gives a tuple
    # bin_mu = numpyro.sample("bin_mu", dist.RelaxedBernoulli(0.5, probs=theta), (n,))   # so seems that this gives a tuple

    # 1d random walk for R
    R_d = numpyro.sample("R_d", dist.GaussianRandomWalk(scale=sigma_R, num_steps=n))
    R = numpyro.deterministic('R',jnp.exp(R_d)) # prevents from negative values

    # 1d random walk for importations
    mu_d = numpyro.sample("mu_d", dist.GaussianRandomWalk(scale=sigma_mu, num_steps=n))
    mu = numpyro.deterministic('mu', jnp.multiply(bin_mu,jnp.exp(mu_d)))

    # mu = numpyro.deterministic('mu', bin_mu * jnp.exp(mu_d))
    # mu = numpyro.deterministic('mu', jnp.exp(mu_d))

    # incidence
    incidence = numpyro.deterministic('incidence',compute_incidence_faster(R,g,mu))

    # observed data
    numpyro.sample('obs', dist.Poisson(incidence), obs=y)

## Fit the model

In [ ]:
def fit(model, obs, g, r_prior=1.0):
  mcmc = MCMC(NUTS(renewal, target_accept_prob=0.8),
              num_chains=4,
              num_warmup=1000,
              num_samples=20000,
              progress_bar=False)
              # num_warmup=00,
              # num_samples=500,
              # progress_bar=True)
  mcmc.run(random.PRNGKey(0), y=jnp.array(obs), g=jnp.array(g), r_prior=r_prior)
  samples = mcmc.get_samples()
  return samples, mcmc


In [ ]:
# r_pior = 0.1
# samples1, mcmc1 = fit(renewal, obs1, g1, r_prior=r_pior)
# with open(path + 'mcmc_1.pkl', 'wb') as f:
#     dill.dump(mcmc1, f)
# samples2, mcmc2 = fit(renewal, obs2, g2, r_prior=r_pior)
# with open(path + 'mcmc_2.pkl', 'wb') as f:
#     dill.dump(mcmc2, f)
# samples3, mcmc3 = fit(renewal, obs3, g3, r_prior=r_pior)
# with open(path + 'mcmc_3.pkl', 'wb') as f:
#     dill.dump(mcmc3, f)
# samples4, mcmc4 = fit(renewal, obs4, g4, r_prior=r_pior)
# with open(path + 'mcmc_4.pkl', 'wb') as f:
#     dill.dump(mcmc4, f)
# samples5, mcmc5 = fit(renewal, obs5, g5, r_prior=r_pior)
# with open(path + 'mcmc_5.pkl', 'wb') as f:
#     dill.dump(mcmc5, f)
# samples6, mcmc6 = fit(renewal, obs6, g6, r_prior=r_pior)
# with open(path + 'mcmc_6.pkl', 'wb') as f:
#     dill.dump(mcmc6, f)
# samples7, mcmc7 = fit(renewal, obs7, g7, r_prior=r_pior)
# with open(path + 'mcmc_7.pkl', 'wb') as f:
#     dill.dump(mcmc7, f)
# samples8, mcmc8 = fit(renewal, obs8, g8, r_prior=r_pior)
# with open(path + 'mcmc_8.pkl', 'wb') as f:
#     dill.dump(mcmc8, f)
# samples9, mcmc9 = fit(renewal, obs9, g9, r_prior=r_pior)
# with open(path + 'mcmc_9.pkl', 'wb') as f:
#     dill.dump(mcmc9, f)
# samples10, mcmc10 = fit(renewal, obs10, g10, r_prior=r_pior)
# with open(path + 'mcmc_10.pkl', 'wb') as f:
#     dill.dump(mcmc10, f)
# samples11, mcmc11 = fit(renewal, obs11, g11, r_prior=r_pior)
# with open(path + 'mcmc_11.pkl', 'wb') as f:
#     dill.dump(mcmc11, f)
# samples12, mcmc12 = fit(renewal, obs12, g12, r_prior=r_pior)
# with open(path + 'mcmc_12.pkl', 'wb') as f:
#     dill.dump(mcmc12, f)


## Plot and compare results

In [ ]:
def plot_sigmas(samples):
  plt.figure(figsize=(15,5))
  plt.subplot(1,3,1)
  plt.hist(samples['sigma_mu'], label = 'posterior')
  # plt.axvline(x = sigma_mu, color='red', label = 'true')
  plt.legend()
  plt.xlabel('sigma_mu')

  plt.subplot(1,3,2)
  plt.hist(samples['sigma_R'], label = 'posterior')
  # plt.axvline(x = sigma_R, color='red', label = 'true')
  plt.legend()
  plt.xlabel('sigma_R')

  # plt.subplot(1,3,3)
  # plt.hist(samples['mult_mu'], label = 'posterior')
  # # plt.axvline(x = 5, color='red', label = 'true')
  # plt.legend()
  # plt.xlabel('mult_mu')

  plt.subplot(1,3,3)
  plt.hist(samples['theta'], label = 'posterior')
  # plt.axvline(x = 5, color='red', label = 'true')
  plt.legend()
  plt.xlabel('theta')

  plt.show()

def plot_posterior(samples, R, mu, inc, title=None):
  plt.figure(figsize=(15,5))

  x = np.arange(n)
  plt.subplot(1,3,1)
  # plt.plot(samples['R'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples['R'], 5, axis=0), np.percentile(samples['R'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples['R'], 50, axis=0), label='post median', color='tab:blue')
  plt.plot(R, color = 'red', alpha=1.0, label = 'true')
  plt.legend()
  plt.ylim(-0.5,14)
  plt.xlabel('days')
  plt.ylabel(r'$R_t$')
  plt.title(r'$R_t$')

  plt.subplot(1,3,2)
  # plt.plot(samples['mu'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples['mu'], 5, axis=0), np.percentile(samples['mu'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples['mu'], 50, axis=0), label='post median', color='tab:blue')
  plt.plot(mu, color = 'red', alpha=1.0, label = 'true')
  plt.legend()
  # plt.ylim(0,np.max((samples['mu'].max(), mu.max()))*1.1)
  # plt.ylim(0, mu.max() * 2)
  plt.xlabel('days')
  plt.ylabel('# importations')
  plt.title('Importations')

  plt.subplot(1,3,3)
  # plt.plot(samples['incidence'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples['incidence'], 5, axis=0), np.percentile(samples['incidence'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples['incidence'], 50, axis=0), label='post median', color='tab:blue')
  plt.plot(inc, color = 'red', alpha=1.0, label = 'true')
  # plt.ylim(0,samples['incidence'].max())
  plt.legend()
  plt.xlabel('days')
  plt.ylabel('# new cases')
  plt.title('Incidence')


  if title is None:
    plt.suptitle('Posterior predictive')
  else:
    plt.suptitle('Posterior predictive, ' + title)
  plt.tight_layout()
  plt.show()

In [ ]:
r_pior = 100.0
# samples1a, mcmc1a = fit(renewal, obs1, g1, r_prior=r_pior)
# with open(path + 'mcmc_1a.pkl', 'wb') as f:
#     dill.dump(mcmc1a, f)
# samples2a, mcmc2a = fit(renewal, obs2, g2, r_prior=r_pior)
# with open(path + 'mcmc_2a.pkl', 'wb') as f:
#     dill.dump(mcmc2a, f)
# samples3a, mcmc3a = fit(renewal, obs3, g3, r_prior=r_pior)
# with open(path + 'mcmc_3a.pkl', 'wb') as f:
#     dill.dump(mcmc3a, f)
# samples4a, mcmc4a = fit(renewal, obs4, g4, r_prior=r_pior)
# with open(path + 'mcmc_4a.pkl', 'wb') as f:
#     dill.dump(mcmc4a, f)
# samples5a, mcmc5a = fit(renewal, obs5, g5, r_prior=r_pior)
# with open(path + 'mcmc_5a.pkl', 'wb') as f:
#     dill.dump(mcmc5a, f)
# samples6a, mcmc6a = fit(renewal, obs6, g6, r_prior=r_pior)
# with open(path + 'mcmc_6a.pkl', 'wb') as f:
#     dill.dump(mcmc6a, f)
# samples7a, mcmc7a = fit(renewal, obs7, g7, r_prior=r_pior)
# with open(path + 'mcmc_7a.pkl', 'wb') as f:
#     dill.dump(mcmc7a, f)
# samples8a, mcmc8a = fit(renewal, obs8, g8, r_prior=r_pior)
# with open(path + 'mcmc_8a.pkl', 'wb') as f:
#     dill.dump(mcmc8a, f)
# samples9a, mcmc9a = fit(renewal, obs9, g9, r_prior=r_pior)
# with open(path + 'mcmc_9a.pkl', 'wb') as f:
#     dill.dump(mcmc9a, f)
samples10a, mcmc10a = fit(renewal, obs10, g10, r_prior=r_pior)
with open(path + 'mcmc_10a.pkl', 'wb') as f:
    dill.dump(mcmc10a, f)
samples11a, mcmc11a = fit(renewal, obs11, g11, r_prior=r_pior)
with open(path + 'mcmc_11a.pkl', 'wb') as f:
    dill.dump(mcmc11a, f)
samples12a, mcmc12a = fit(renewal, obs12, g12, r_prior=r_pior)
with open(path + 'mcmc_12a.pkl', 'wb') as f:
    dill.dump(mcmc12a, f)

# Show in one plot

In [ ]:
def plot_posterior_two_samples(samples1, samples2, R, mu, inc, title=None):
  plt.figure(figsize=(20,5))

  x = np.arange(n)
  plt.subplot(1,4,1)
  # plt.plot(samples['R'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples1['R'], 5, axis=0), np.percentile(samples1['R'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples1['R'], 50, axis=0), label='post median high var', color='tab:blue')
  plt.plot(R, color = 'red', alpha=1.0, label = 'true')
  plt.legend()
  plt.ylim(-0.5,14)
  plt.xlabel('days')
  plt.ylabel(r'$R_t$')
  plt.title(r'$R_t$ high var')

  plt.subplot(1,4,2)
  # plt.plot(samples['mu'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples1['mu'], 5, axis=0), np.percentile(samples1['mu'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples1['mu'], 50, axis=0), label='post median high var', color='tab:blue')
  plt.plot(mu, color = 'red', alpha=1.0, label = r'$\mu$')
  plt.plot(inc, color = 'orange', alpha=1.0, label = 'total incidence')
  plt.legend()
  # plt.ylim(0,np.max((samples['mu'].max(), mu.max()))*1.1)
  # plt.ylim(0, mu.max() * 2)
  plt.xlabel('days')
  plt.ylabel('# new cases')
  plt.title('Importations high var')

  plt.subplot(1,4,3)
  # plt.plot(samples['R'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples2['R'], 5, axis=0), np.percentile(samples2['R'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples2['R'], 50, axis=0), label='post median low var', color='tab:blue')
  plt.plot(R, color = 'red', alpha=1.0, label = 'true')
  plt.legend()
  plt.ylim(-0.5,14)
  plt.xlabel('days')
  plt.ylabel(r'$R_t$')
  plt.title(r'$R_t$ low var')

  plt.subplot(1,4,4)
  # plt.plot(samples['mu'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples2['mu'], 5, axis=0), np.percentile(samples2['mu'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples2['mu'], 50, axis=0), label='post median low var', color='tab:blue')
  plt.plot(mu, color = 'red', alpha=1.0, label = r'$\mu$')
  plt.plot(inc, color = 'orange', alpha=1.0, label = 'total incidence')
  plt.legend()
  # plt.ylim(0,np.max((samples['mu'].max(), mu.max()))*1.1)
  # plt.ylim(0, mu.max() * 2)
  plt.xlabel('days')
  plt.ylabel('# new cases')
  plt.title('Importations low var')


  # plt.subplot(1,3,3)
  # # plt.plot(samples['incidence'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  # plt.fill_between(x, np.percentile(samples['incidence'], 5, axis=0), np.percentile(samples['incidence'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  # plt.plot(x, np.percentile(samples['incidence'], 50, axis=0), label='post median', color='tab:blue')
  # plt.plot(inc, color = 'red', alpha=1.0, label = 'true')
  # # plt.ylim(0,samples['incidence'].max())
  # plt.legend()
  # plt.xlabel('days')
  # plt.ylabel('# new cases')
  # plt.title('Incidence')


  if title is None:
    plt.suptitle('Posterior predictive')
  else:
    plt.suptitle(title)
  plt.tight_layout()
  plt.show()

In [ ]:
# plot_posterior_two_samples(samples1, samples10, R1, mu1, inc1, title=scenarios_names[1])
# plot_posterior_two_samples(samples2, samples20, R2, mu2, inc2, title=scenarios_names[2])
# plot_posterior_two_samples(samples3, samples30, R3, mu3, inc3, title=scenarios_names[3])
# plot_posterior_two_samples(samples4, samples40, R4, mu4, inc4, title=scenarios_names[4])
# plot_posterior_two_samples(samples5, samples50, R5, mu5, inc5, title=scenarios_names[5])
# plot_posterior_two_samples(samples6, samples60, R6, mu6, inc6, title=scenarios_names[6])
# plot_posterior_two_samples(samples7, samples70, R7, mu7, inc7, title=scenarios_names[7])
# plot_posterior_two_samples(samples8, samples80, R8, mu8, inc8, title=scenarios_names[8])
# plot_posterior_two_samples(samples9, samples90, R9, mu9, inc9, title=scenarios_names[9])


In [ ]:
ls /content/drive/MyDrive/Colab\ Notebooks/importations/Data/

Double_spike/  Seasonal/  sim_double_spike_mu.csv  sim_double_spike_Rt.csv  sim_double_spike_y.csv


# Test different priors for R_t variance

In [ ]:
assert(1==2)

AssertionError: ignored

In [ ]:
r_var = [0.01, 0.1, 0.5, 1.0, 2.5, 5.0, 7.5, 10.0, 20.0, 50.0]
samples_dict = {}

In [ ]:
for rv in r_var:
  print(rv)
  s = fit(renewal, obs, g, r_prior=rv)
  samples_dict.update({rv: s})

In [ ]:
for rv in r_var:
  s = samples_dict[rv]
  plot_posterior(s, 'R_t variance ~ Exp(' + str(rv) + ')')

# Plot in one image


In [ ]:
def plot_posterior(samples, title=None):
  plt.figure(figsize=(15,5))

  x = np.arange(n)
  plt.subplot(1,3,1)
  # plt.plot(samples['R'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples['R'], 5, axis=0), np.percentile(samples['R'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples['R'], 50, axis=0), label='post median', color='tab:blue')
  plt.plot(R, color = 'red', alpha=1.0, label = 'true')
  # plt.legend()
  plt.ylim(-0.5,14)
  plt.xlabel('days')
  plt.ylabel(r'$R_t$')
  plt.title(r'$R_t$')

  plt.subplot(1,3,2)
  # plt.plot(samples['mu'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples['mu'], 5, axis=0), np.percentile(samples['mu'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples['mu'], 50, axis=0), label='post median', color='tab:blue')
  plt.plot(mu, color = 'red', alpha=1.0, label = 'true')
  # plt.legend()
  # plt.ylim(0,np.max((samples['mu'].max(), mu.max()))*1.1)
  # plt.ylim(0, mu.max() * 2)
  plt.xlabel('days')
  plt.ylabel('# importations')
  plt.title('Importations')

  plt.subplot(1,3,3)
  # plt.plot(samples['incidence'][0::100,:].T, color = 'b', alpha=0.5, label = 'posterior')
  plt.fill_between(x, np.percentile(samples['incidence'], 5, axis=0), np.percentile(samples['incidence'], 95, axis=0), alpha=0.3, label='CrI', color='tab:blue')
  plt.plot(x, np.percentile(samples['incidence'], 50, axis=0), label='posterior median', color='tab:blue')
  plt.plot(inc, color = 'red', alpha=1.0, label = 'true')
  # plt.ylim(0,samples['incidence'].max())
  plt.legend()
  plt.xlabel('days')
  plt.ylabel('# new cases')
  plt.title('Incidence')


  if title is None:
    plt.suptitle('Posterior predictive')
  else:
    plt.suptitle('Posterior predictive, ' + title)
  plt.tight_layout()
  plt.show()

In [ ]:
samples_dict.keys()

In [ ]:
r_var_plots = [0.01, 0.1, 0.5, 1.0, 2.5, 5.0, 7.5, 10.0, 20.0, 50.0]

In [ ]:
for rv in r_var:
  s = samples_dict[rv]
  plot_posterior(s, 'R_t variance ~ Exp(' + str(rv) + ')')